# ABOUT


Datascientest's Datascientist continuous bootcamp - cohorte Mars2022 -  AeroBOT project

**Tutor**

* Alban THUET

**Authors:**

* Hichem HADJI  

</br>

---
</br>

**Version History**

Version | Date       | Author(s)  | Modification
--------|----------- | ---------  | --------------------------
1.0     | 20/10/2022 | H.H       | modif

This notebook can be executed entirely. 

It

* mounts the GDrive of our AeroBot project @gmail account.

* loads the data from the `df_for_Anomaly_prediction.pkl` under a pandas DataFrame named `df`, which contains `96986` entries and 20 columns.
These data do not contain any more UAS-related entries.
* A comparison of 2 naive_bayes approaches while using 2 different vectorizers (Count_Vectorizer vs tfidf_vectorizer) while working with different max_features (500, 1000, 3000) 

# IMPORT PACKAGES


settings for  full / patial Narrative display. Helene?


In [ ]:
#######################
# Import packages
#######################
import numpy as np
import seaborn as sns

#######################
# Pandas
#######################
import pandas as pd
# Set pandas settings to show all data when using .head(), .columns etc.
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option("display.colheader_justify","left") # left-justify the print output of pandas

### Display full columnwidth
# Set pandas settings to display full text columns
#pd.options.display.max_colwidth = None
# Restore pandas settings to display standard colwidth
pd.reset_option('display.max_colwidth')

import itertools # Pour créer des iterateurs

# Package to show the progression of pandas operations
from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks

# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()
# simply use .progress_apply() instead of .apply() on your pd.DataFram

######################
# PLOTTING
######################
import matplotlib.pyplot as plt
%matplotlib inline
# Define global plot parameters for better readability and consistency among plots
# A complete list of the rcParams keys can be retrieved via plt.rcParams.keys() function
plt.rcParams['axes.titlesize'] = 30
plt.rcParams['axes.labelsize'] = 23
plt.rcParams['xtick.labelsize'] = 23
plt.rcParams['ytick.labelsize'] = 23
plt.rc('legend', fontsize=23)    # legend fontsize

# BOKEH 
from bokeh.plotting import figure # Importation de la classe figure qui permet de créer un graphique bokeh.
from bokeh.io import  push_notebook, output_notebook, show
output_notebook() # permet d'afficher tous les futurs graphiques dans l'output d'une cellule jupyter. Si cette instruction n'est pas lancée, la figure s'affichera dans un nouvel onglet.
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
from bokeh.models.tools import HoverTool

#####################
# NLP 
#####################
import re # for Regular Expression handling
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # WordNet lemmatizer
nltk.download('omw-1.4') # necessary for WordNet lemmatizer
from nltk.tokenize import word_tokenize # Usual tokenizer
from nltk.tokenize import TweetTokenizer # Special tokenizer;  "we'll", "didn't", etc. are considered as one word
from sklearn.feature_extraction.text import CountVectorizer # Vectorization
from nltk.corpus import stopwords # Import stopwords from nltk.corpus
from nltk.stem.snowball import EnglishStemmer

######################
# Naive bayes
######################
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier

##############################
# Undersampeling
##############################
from imblearn.under_sampling import RandomUnderSampler 

###############################
# ML preprocessing and models
###############################
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import ensemble # random forest
from sklearn.svm import SVC

from sklearn.metrics import classification_report, confusion_matrix

import pickle as pkl

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


# LOAD DATA

## Mount GDrive

In [ ]:
#@title
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive/')

#check your present working directory 
%pwd

Mounted at /content/drive/


'/content'

In [ ]:
#@title
# move to the transformed data location (you can create a deeper structure, if needed, e.g. to save a trained model):
%cd /content/drive/MyDrive/data/transformed/

/content/drive/MyDrive/data/transformed


In [ ]:
#@title
!ls # list the content of the pwd

#!ls "/content/drive/MyDrive/Data_Science/Formations/DataScienceTest/projet/AeroBot/" # list contect of a speficic folder

 2022_09_11_7_4_3_raw_narr_BERT_BASE_frozen_max_length_345.pkl
 complaints-2022-08-05_13_55.csv
'Copy of Qualified abbreviations_20220718.xlsx.gsheet'
'Data Dictionnary.xlsx'
 data_for_BERT_multilabel_20220805.pkl
 df_for_Anomaly_prediction.pkl
 model.png
 model_results
 Narrative_PP_stemmed_24072022_TRAIN.pkl
 Narrative_Raw_Stemmed_24072022_TRAIN.pkl
 Narrative_RegEx_subst_21072022_TRAIN.pkl
'Qualified abbreviations_20220707_test.csv'
'Qualified abbreviations_20220708.csv'
'Qualified abbreviations_20220718.csv'
'Qualified abbreviations_20220718_Google_sheet.gsheet'
 test_data_final.pkl
 train_data_final.pkl


## Load data from .pkl file


In [ ]:
# Load the TRAIN data (97417 entries)
# Do not touch the TEST data until the end of the project!
# or the curse of the greek gods will fall upon you!

%cd /content/drive/MyDrive/data/transformed/
with open("df_for_Anomaly_prediction.pkl", "rb") as f:
    loaded_data = pkl.load(f)

df = loaded_data
print("\nA Dataframe with", len(df), "entries has been loaded")

/content/drive/MyDrive/data/transformed


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96986 entries, 1163382 to 874642
Data columns (total 20 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   Narrative                                     96986 non-null  object
 1   Anomaly                                       96986 non-null  object
 2   Narrative_PP_stemmed                          96986 non-null  object
 3   Narrative_Raw_Stemmed                         96986 non-null  object
 4   Narrative_Raw_Stemmed_str                     96986 non-null  object
 5   Narrative_PP_stemmed_str                      96986 non-null  object
 6   Anomaly_Deviation / Discrepancy - Procedural  96986 non-null  int64 
 7   Anomaly_Aircraft Equipment                    96986 non-null  int64 
 8   Anomaly_Conflict                              96986 non-null  int64 
 9   Anomaly_Inflight Event / Encounter            96986 non-null  int

In [ ]:
# Retriece the list of Anomaly label columns
Anomaly_RootLabels_columns = []

for col in df.columns:
  if 'Anomaly_' in str(col):
      Anomaly_RootLabels_columns.append(col)

In [ ]:
Anomaly_RootLabels_columns

['Anomaly_Deviation / Discrepancy - Procedural',
 'Anomaly_Aircraft Equipment',
 'Anomaly_Conflict',
 'Anomaly_Inflight Event / Encounter',
 'Anomaly_ATC Issue',
 'Anomaly_Deviation - Altitude',
 'Anomaly_Deviation - Track / Heading',
 'Anomaly_Ground Event / Encounter',
 'Anomaly_Flight Deck / Cabin / Aircraft Event',
 'Anomaly_Ground Incursion',
 'Anomaly_Airspace Violation',
 'Anomaly_Deviation - Speed',
 'Anomaly_Ground Excursion',
 'Anomaly_No Specific Anomaly Occurred']

# PREDICTION

In [ ]:
def Naive_bayes_count(data, target, max_features):
  """
  Return the predictions metrics for Naive bayes

  Inputs:
  - data: pd.Series containing the narratives as single string
  - target: ndarray of a shape(#samples, #classes) containing 0's and 1's
            A sample may contain several 1's (multilabel context)
  - max_features : Consider the top max_features ordered by term frequency across the corpus.

  Returns: 
  - a classification report in a form of dataframe displaing the different metrics 
    and their values for each label

  """
  
  # Train-test split  
  X_train, X_test, y_train, y_test = train_test_split(data, target, 
                                                      test_size= 0.2, 
                                                      random_state = 12)
  
  d = pd.DataFrame()
  
  # Define a pipeline combining a text feature extractor with multi lable classifier
  NB_pipeline = Pipeline([
      ('vect', CountVectorizer(token_pattern=r"\b[a-zA-Z][a-zA-Z]+\b",
                                  max_features = max_features)),
                      ('clf', OneVsRestClassifier(MultinomialNB(
                          fit_prior=True, class_prior=None))),
                  ])
  
  # Train the model using X_train and y_train_Anomaly
  NB_pipeline.fit(X_train,y_train)
  
  # Compute the testing accuracy 
  y_pred = NB_pipeline.predict(X_test)
    
  # classification report 
  clf_rep = classification_report(y_test, y_pred, output_dict=True)
  
  # write classification report dictionnary into pd.DataFrame
  metrics = pd.DataFrame(clf_rep)
  
  # The rest of the code is basically kind of 'transposing' the format 
  # and adding extra columns with parameter values

  # Rename columns with anomaly names
  # Crete dictionary with correspondance among label indices and anomaly names
  anomaly_labels = dict(zip(metrics.columns[0:14], Anomaly_RootLabels_columns))
  metrics = metrics.rename(columns = anomaly_labels)

  # Create DataFrame in the right format for the plotting of results
  clf_rep_df = pd.DataFrame()
  for anomaly in metrics.columns[0:14]:

    temp_df = pd.DataFrame(index = metrics.index) # create temporary DataFrame with the 4 metrics as index
    temp_df['values'] = metrics.filter(items = [anomaly]).values # write the 4 values for the selected anomaly
    temp_df['anomaly'] = anomaly # fill in the column with the selected anomaly label
    clf_rep_df = pd.concat([clf_rep_df, temp_df])

  clf_rep_df = clf_rep_df.reset_index().rename(columns = {'index': 'metric'})

  # Fill in additionnal columns with metadata
  clf_rep_df['classifier'] = 'naive bayes'  # [Word_Embedding, BERT...]
  clf_rep_df['preprocessing'] = 1              # 0 if raw, 1 if preprocessed
  clf_rep_df['undersampling'] = 0 
  clf_rep_df['num_words'] = max_features
  clf_rep_df['vectorizer'] = 'count'     
  
  # Reorder columns
  clf_rep_df = clf_rep_df[['classifier','vectorizer', 'preprocessing', 'undersampling',
                      'anomaly', 'num_words', 'metric', 'values']]
  

  return clf_rep_df

In [ ]:
def Naive_bayes_tidf(data, target, max_features):
  """
  Return the predictions metrics for Naive bayes

  Inputs:
  - data: pd.Series containing the narratives as single string
  - target: ndarray of a shape(#samples, #classes) containing 0's and 1's
            A sample may contain several 1's (multilabel context)
  - max_features : Consider the top max_features ordered by term frequency across the corpus.

  Returns: 
  - a classification report in a form of dataframe displaing the different metrics 
    and their values for each label

  """
  
  # Train-test split  
  X_train, X_test, y_train, y_test = train_test_split(data, target, 
                                                      test_size= 0.2, 
                                                      random_state = 12)
  
  d = pd.DataFrame()
  
  # Define a pipeline combining a text feature extractor with multi lable classifier
  NB_pipeline = Pipeline([
      ('tidf', TfidfVectorizer(token_pattern=r"\b[a-zA-Z][a-zA-Z]+\b",
                                  max_features = max_features)),
                      ('clf', OneVsRestClassifier(MultinomialNB(
                          fit_prior=True, class_prior=None))),
                  ])
  
  # Train the model using X_train and y_train_Anomaly
  NB_pipeline.fit(X_train,y_train)
  
  # Compute the testing accuracy 
  y_pred = NB_pipeline.predict(X_test)
    
  # classification report 
  clf_rep = classification_report(y_test, y_pred, output_dict=True)
  
  # write classification report dictionnary into pd.DataFrame
  metrics = pd.DataFrame(clf_rep)
  
  # The rest of the code is basically kind of 'transposing' the format 
  # and adding extra columns with parameter values

  # Rename columns with anomaly names
  # Crete dictionary with correspondance among label indices and anomaly names
  anomaly_labels = dict(zip(metrics.columns[0:14], Anomaly_RootLabels_columns))
  metrics = metrics.rename(columns = anomaly_labels)

  # Create DataFrame in the right format for the plotting of results
  clf_rep_df = pd.DataFrame()
  for anomaly in metrics.columns[0:14]:

    temp_df = pd.DataFrame(index = metrics.index) # create temporary DataFrame with the 4 metrics as index
    temp_df['values'] = metrics.filter(items = [anomaly]).values # write the 4 values for the selected anomaly
    temp_df['anomaly'] = anomaly # fill in the column with the selected anomaly label
    clf_rep_df = pd.concat([clf_rep_df, temp_df])

  clf_rep_df = clf_rep_df.reset_index().rename(columns = {'index': 'metric'})

  # Fill in additionnal columns with metadata
  clf_rep_df['classifier'] = 'naive bayes'  # [Word_Embedding, BERT...]
  clf_rep_df['preprocessing'] = 1              # 0 if raw, 1 if preprocessed
  clf_rep_df['undersampling'] = 0 
  clf_rep_df['num_words'] = max_features
  clf_rep_df['vectorizer'] = 'tfidf'     
  
  # Reorder columns
  clf_rep_df = clf_rep_df[['classifier','vectorizer', 'preprocessing', 'undersampling',
                      'anomaly', 'num_words', 'metric', 'values']]
  

  return clf_rep_df

In [ ]:
# initialize the objects necesseray function inputs
data = df['Narrative_PP_stemmed_str']
target = df[Anomaly_RootLabels_columns].values

#Generate the classification report tabels for both Countvectorizer and Tfidf vectorizer
naive_bayes_count_500 = Naive_bayes_count(data, target,max_features = 500)
naive_bayes_count_1000 = Naive_bayes_count(data, target,max_features = 1000)
naive_bayes_count_3000 = Naive_bayes_count(data, target,max_features = 3000)

naive_bayes_tfidf_500 = Naive_bayes_tidf(data, target,max_features = 500)
naive_bayes_tfidf_1000 = Naive_bayes_tidf(data, target,max_features = 1000)
naive_bayes_tfidf_3000 = Naive_bayes_tidf(data, target,max_features = 3000)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'NBayes_COUNTVEC_PP_NWORDS_3000.pkl'
#enregistre le modèle
pkl.dump([naive_bayes_count_3000], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'NBayes_COUNTVEC_PP_NWORDS_1000.pkl'
#enregistre le modèle
pkl.dump([naive_bayes_count_1000], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'NBayes_COUNTVEC_PP_NWORDS_500.pkl'
#enregistre le modèle
pkl.dump([naive_bayes_count_500], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'NBayes_TIDFVEC_PP_NWORDS_500.pkl'
#enregistre le modèle
pkl.dump([naive_bayes_tfidf_500], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'NBayes_TIDFVEC_PP_NWORDS_1000.pkl'
#enregistre le modèle
pkl.dump([naive_bayes_tfidf_1000], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'NBayes_TIDFVEC_PP_NWORDS_3000.pkl'
#enregistre le modèle
pkl.dump([naive_bayes_tfidf_3000], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results
